# Saliva Processing

This notebook processes saliva data from "raw" format, as coming from the laboratory, into a more "machine-friendly" format.

In [ ]:
import json
from pathlib import Path

import pandas as pd

import biopsykit as bp

from fau_colors import cmaps
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

palette

## Setup Paths

In [ ]:
deploy_type = "local"

In [ ]:
config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))
base_path = Path(config_dict[deploy_type]["base_path"])

saliva_path = base_path.joinpath("saliva_total")

In [ ]:
output_path = saliva_path.joinpath("processed")
bp.utils.file_handling.mkdirs(output_path)

In [ ]:
sample_times = [-20, -1, 0, 10, 20, 45]

## Cortisol

### Load and Rename

In [ ]:
cort_data = bp.io.saliva.load_saliva_plate(
    saliva_path.joinpath("raw/cortisol_samples_raw.xlsx"),
    saliva_type="cortisol",
    id_col_names=["subject", "condition", "sample"],
    regex_str="VPN(\d+)_(FTSST|TSST)_(\d)",
)
vpn_names = cort_data.index.get_level_values("subject").unique()
# rename subject IDs
cort_data = cort_data.rename(index={old: "VP_{:02d}".format(int(old)) for old in vpn_names}, level="subject")
cort_data = cort_data.rename(index={"FTSST": "ftsst", "TSST": "tsst"}, level="condition")

cort_data

### Compute Features

In [ ]:
max_inc = bp.saliva.max_increase(cort_data, "cortisol", remove_s0=True)
max_inc_percent = bp.saliva.max_increase(cort_data, "cortisol", remove_s0=True, percent=True)
auc = bp.saliva.auc(cort_data, "cortisol", remove_s0=True, sample_times=sample_times)
slope = bp.saliva.slope(cort_data, sample_idx=[1, 4], saliva_type="cortisol", sample_times=sample_times)


cort_features = pd.concat([max_inc, max_inc_percent, auc, slope], axis=1)
cort_features = bp.saliva.utils.saliva_feature_wide_to_long(cort_features, saliva_type="cortisol")
cort_features.head()

### Save

In [ ]:
bp.io.saliva.save_saliva(output_path.joinpath("cortisol_samples.csv"), cort_data, "cortisol")
cort_features.to_csv(output_path.joinpath("cortisol_features.csv"))